# Colander Benchmarking

In [11]:
from colander.mock_data_generation.utils import *
from colander.estimate import greedy_strain_estimation
import networkx as nx

Broadly speaking, this process involves a few steps:

1. Generate a random "starting genome"
2. Generate strains
3. Shear strains into k-mers then make de Bruijn graph
4. Run greedy strain estimation code

Of course, genomes in practice are not random sequences of nucleotides -- as chapter 1 of Compeau and Pevzner shows, factors like G/C skew and repetitive regions are examples of nonrandomness in real genomes. This gives us reason to doubt the efficacy of modeling genomes completely randomly, as we do here.

That being said, we have to start somewhere.

## Define parameter sets for how the tests we're going to run

The parameters are:

- Starting genome length
- Strain coverages
- *N* parameter (each test runs estimation once for each *N* parameter in its list)

(We've kept the k-mer size and hypervariable region settings / mutation rates consistent, but these could of course be adjusted on a per-test basis as well.)

In [12]:
tests = [
    [1000, [1, 3, 5], [1, 2, 3, 4, 5]],
    [1000, [20, 20, 45], [1, 2, 3, 4, 5]],
    [1000, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [5, 10, 15, 20]],
    [10000, [1, 3, 5], [1, 2, 3, 4, 5]],
    [10000, [20, 20, 45], [1, 2, 3, 4, 5]],
    [10000, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [5, 10, 15, 20]]
]

In [13]:
for i in range(len(tests)):
    print("PARAMETER SET {}".format(i))
    print("----------------".format(i))
    glen = tests[i][0]
    genome = generate_random_sequence(glen)
    
    # Define hypervariable regions in the genome: these will undergo more mutations
    hv_regions = [(glen // 50, glen // 50 + 100), (glen // 2, glen // 2 + 100)]

    strains = generate_strains_from_genome(
        genome,
        tests[i][1],
        hv_regions,
        hypervariable_mutation_probability=0.01,
        normal_mutation_probability=0.001
    )
    
    kmers = []
    for s in strains:
        kmers += shear_into_kmers(s.seq, s.coverage, 15)

    g = make_debruijn_graph(kmers)

    for n in tests[i][2]:
        cs = greedy_strain_estimation(g, n)
        print("CycleSet with N = {} has {} cycles and conformity score {}".format(
            n, len(cs), cs.conformity_score(g)
        ))

PARAMETER SET 0
----------------
CycleSet with N = 1 has 1 cycles and conformity score 15237
CycleSet with N = 2 has 2 cycles and conformity score 999
CycleSet with N = 3 has 3 cycles and conformity score 0
CycleSet with N = 4 has 3 cycles and conformity score 0
CycleSet with N = 5 has 3 cycles and conformity score 0
PARAMETER SET 1
----------------
CycleSet with N = 1 has 1 cycles and conformity score 1551600
CycleSet with N = 2 has 2 cycles and conformity score 400000
CycleSet with N = 3 has 3 cycles and conformity score 0
CycleSet with N = 4 has 3 cycles and conformity score 0
CycleSet with N = 5 has 3 cycles and conformity score 0
PARAMETER SET 2
----------------
CycleSet with N = 5 has 5 cycles and conformity score 1004
CycleSet with N = 10 has 6 cycles and conformity score 0
CycleSet with N = 15 has 6 cycles and conformity score 0
CycleSet with N = 20 has 6 cycles and conformity score 0
PARAMETER SET 3
----------------
CycleSet with N = 1 has 1 cycles and conformity score 152520
